In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
import numpy as np
from sklearn.metrics import accuracy_score
from collections import Counter
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D
from keras.layers import Activation, MaxPooling2D, Dropout, Flatten, Reshape
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

def load_data():
  import gdown
  !wget -O cifar_data https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/AI%20Scholars/Sessions%201%20-%205/Session%204%20_%205%20-%20Neural%20Networks%20_%20CNN/dogs_v_roads

  import pickle
  data_dict = pickle.load(open( "cifar_data", "rb" ));
  
  data   = data_dict['data']
  labels = data_dict['labels']
  
  return data, labels

def plot_one_image(data, labels, img_idx):
  from google.colab.patches import cv2_imshow
  import cv2
  import matplotlib.pyplot as plt
  my_img   = data[img_idx, :].squeeze().reshape([32,32,3]).copy()
  my_label = labels[img_idx]
  print('label: %s'%my_label)
  fig, ax = plt.subplots(1,1)

  img = ax.imshow(my_img, extent=[-1,1,-1,1])

  x_label_list = [0, 8, 16, 24, 32]
  y_label_list = [0, 8, 16, 24, 32]

  ax.set_xticks([-1, -0.5, 0, 0.5, 1])
  ax.set_yticks([-1, -0.5, 0, 0.5, 1])

  ax.set_xticklabels(x_label_list)
  ax.set_yticklabels(y_label_list)

  fig.show(img)

def categorical_to_numpy(labels_in):
  return np.array(list(map(lambda x: np.array([1, 0]) if x == "dog" else np.array([0, 1]), labels_in)), dtype="float32")
  
def CNNClassifier(num_epochs=30):
  def create_model():
    model = Sequential()
    model.add(Reshape((32, 32, 3)))
    model.add(Conv2D(32, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    opt = keras.optimizers.RMSprop(learning_rate=0.0001, decay=1e-6)

    model.compile(loss='categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    return model
  return KerasClassifier(build_fn=create_model, epochs=num_epochs, batch_size=10, verbose=2)


def plot_acc(history, ax = None, xlabel = 'Epoch #'):
    history = history.history
    history.update({'epoch':list(range(len(history['val_accuracy'])))})
    history = pd.DataFrame.from_dict(history)

    best_epoch = history.sort_values(by = 'val_accuracy', ascending = False).iloc[0]['epoch']

    if not ax:
      f, ax = plt.subplots(1,1)
    sns.lineplot(x = 'epoch', y = 'val_accuracy', data = history, label = 'Validation', ax = ax)
    sns.lineplot(x = 'epoch', y = 'accuracy', data = history, label = 'Training', ax = ax)
    ax.axhline(0.5, linestyle = '--',color='red', label = 'Chance')
    ax.axvline(x = best_epoch, linestyle = '--', color = 'green', label = 'Best Epoch')  
    ax.legend(loc = 7)    
    ax.set_ylim([0.4, 1])

    ax.set_xlabel(xlabel)
    ax.set_ylabel('Accuracy (Fraction)')
    
    plt.show()

In [ ]:
data, labels = load_data()

In [ ]:
plot_one_image(data, labels, 0) 

In [ ]:
plot_one_image(data, labels, 700) 

In [ ]:
print (len(data))
print(Counter(labels))

In [ ]:
for i in range(0, 5):
  plot_one_image(data, labels, i)

for i in range(700, 705):
  plot_one_image(data, labels, i)

In [ ]:
print('One image looks like: ')
print(data[0])
print("Length of list: ", len(data[0]))

In [ ]:
img_height =  32
img_width =  32
color_channels =  3

if img_height == 32 and img_width == 32 and color_channels == 3:
  print("Correct!")
  print ("Each image is", img_height, 'x', img_width, 'pixels.')
  print ("Each pixel has", color_channels, "channels for red, green, blue.")
  print ("This gives a total of", img_height * img_width * color_channels, "intensity values per image.")
else:
  print("Those aren't quite the values.")
  print("Your values give a total of", img_height * img_width * color_channels, "intensity values per image.") 
  print("Discuss with your group and try again!")

In [ ]:
print ('Data shape:', data.shape)
print ('Data:', data)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, labels, test_size=0.2, random_state=1)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, y_train)

In [ ]:
predictions = knn.predict(X_test)

In [ ]:
print("KNN Testing Set Accuracy:")
print(accuracy_score(y_test, predictions)*100)

In [ ]:
image_id = 210 

plot_one_image(X_test, y_test, image_id)

print('prediction:', knn.predict([X_test[image_id]])[0])

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

knn.fit(X_train, y_train)

predictions = knn.predict(X_test)

knn_acc = accuracy_score(y_test, predictions)

print("KNN Testing Set Accuracy:")
print(knn_acc*100)

In [ ]:
for i in np.arange(1,10,1):
  
  knn = KNeighborsClassifier(n_neighbors=int(i))

  knn.fit(X_train, y_train)
  
  predictions = knn.predict(X_test)

  print("KNN Testing Set Accuracy for %d neighbors:"%i)
  print(accuracy_score(y_test, predictions)*100)

In [ ]:
nnet = MLPClassifier(hidden_layer_sizes=(10, 5, 4), random_state=1, max_iter= 10000)  ## How many hidden layers? How many neurons does this have?
nnet.fit(X_train, y_train)

predictions = nnet.predict(X_test)

print("MLP Testing Accuracy:")
print(accuracy_score(y_test, predictions)*100)

In [ ]:
for layers in [(1), (1,1), (10), (10,5)]: 

  print('Layer parameters are ...')
  print(layers)
  nnet = MLPClassifier(hidden_layer_sizes=layers, random_state=1, max_iter=1000) 
  
  nnet.fit(X_train, y_train)

  predictions = nnet.predict(X_test)

  print("MLP Testing Accuracy:")
  print(accuracy_score(y_test, predictions)*100)  

In [ ]:
X_train = X_train.astype(float)
X_test = X_test.astype(float)

cnn = CNNClassifier()
cnn.fit(X_train, y_train)

predictions = cnn.predict(X_test)

print("CNN Testing Accuracy:")
print(accuracy_score(y_test, predictions)*100)

In [ ]:
cnn = CNNClassifier(num_epochs=80)

history = cnn.fit(X_train, categorical_to_numpy(y_train), validation_data=(X_test, categorical_to_numpy(y_test)))

plot_acc(history)